In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import re
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup as bs

from selenium import webdriver

import time

import warnings
warnings.filterwarnings('ignore')

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

url = 'http://www.riss4u.net/search/detail/DetailView.do?p_mat_type=3a11008f85f7c51d&control_no=5f647ece99a4952d&keyword=%ED%8A%B9%EC%88%98%EC%B2%B4%EC%9C%A1%ED%95%99%ED%9A%8C%EC%A7%80'
driver = webdriver.Chrome('chromedriver', options=options)
driver.get(url)

time.sleep(1)
driver.implicitly_wait(10)

####################연도박스활성화
year_box = driver.find_element_by_xpath('//*[@id="divContent"]/div[1]/div/div[2]/ul')

####################권호 url추출
book_urls = year_box.find_elements_by_xpath(".//li[@id]/a")
book_url_list = []
for u in book_urls:
  book_url_list.append(u.get_attribute('href'))


print('한국특수체육학회지는 1993년부터 2021년까지 {}권이 출간되었습니다'.format(len(book_urls)))
print('따라서 링크의 개수는 {}개 입니다'.format(len(book_urls)), '\n')

####################권호별 링크 들어가서 정보수집
real_list = []

for i, book_url in enumerate(book_url_list[60:75]): #83, 88, 89제외

  print('{}번째 링크입니다'.format(i+1))
  
  driver = webdriver.Chrome('chromedriver', options=options)
  driver.get(book_url)
  time.sleep(1)
  driver.implicitly_wait(1)
  
  #권호 확인
  book_no = driver.find_element_by_xpath('//*[@id="soptionview"]/div/div[2]/a').text
  
  #100개씩 보기 및 조회 클릭
  driver.find_element_by_xpath('//*[@id="soptionview"]/div/div[3]/div[1]/div[2]/div/div[1]').click()
  driver.find_element_by_xpath('//*[@id="soptionview"]/div/div[3]/div[1]/div[2]/div/div[2]/div/ul/li[5]/a').click()
  driver.find_element_by_xpath('//*[@id="soptionview"]/div/div[3]/div[1]/div[2]/button').click()
      
  #제목 url수집
  title_box = driver.find_element_by_xpath('//*[@id="soptionview"]/div/div[3]/div[2]/ul') #내용 들어있는 곳 활성화
  titles= title_box.find_elements_by_xpath('.//li/div/p/a') #제목의 a태그 모두 수집
  title_url = [title.get_attribute('href') for title in titles]
      
  print('{} 논문의 갯수는 {}개 입니다.'.format(book_no,len(title_url)))
  
  
  for url in title_url:
    need_list = []

    #제목 url로 접속
    driver = webdriver.Chrome('chromedriver', options=options)
    driver.get(url)
    
    #접속한 페이지 크롤링준비
    html = driver.page_source
    soup = bs(html, 'lxml')
    
    #추출할정보1(제목)
    titl = soup.find('h3', attrs={'class': 'title'}).get_text().strip()
    titl = re.sub('[\n\t]', '', titl)
    need_list.append(titl.strip())

    #필요부분 추출을 위한 활성화
    need_box = soup.find('div', attrs={'class': 'infoDetailL'})
    rows =  need_box.find_all('li')
    # ro = [row.find('p').get_text().strip() for row in rows]
    rowlist = [row.find('span', attrs={'class': 'strong'}).get_text() for row in rows] # 저자, 권호사항, 주제어 등등

    #추출할정보2
    need_sector = ['저자', '권호사항', '주제어', '수록면']

    for need in need_sector:
      if need in rowlist:    
        ro = [row.find('p').get_text().strip() for row in rows if row.find('span', attrs={'class': 'strong'}).get_text() == need]
        for r in ro:
          need_list.append(re.sub('[\n\t\xa0]', '', r))
      else:
        need_list.append('정보없음')


    #초록-> 2개가(국문, 영문) 있다는 가정
    abstracts = soup.select('div.text.off')
    #초록없을때
    if len(abstracts) == 0:
        need_list.append('초록없음')
        need_list.append('초록없음')
    #초록1개일때
    elif len(abstracts) == 1:
        need_list.append(abstracts[0].p.get_text())
        need_list.append('초록없음')
    #초록2개일때
    elif len(abstracts) == 2:
        need_list.append(abstracts[0].p.get_text())
        need_list.append(abstracts[1].p.get_text())
    #참고문헌있을때
    elif len(abstracts) == 3:
        need_list.append(abstracts[0].p.get_text())
        need_list.append(abstracts[1].p.get_text())
          
    real_list.append(need_list)            
      
  print('{}번째 링크의 크롤링이 완료되었습니다'.format(i+1), '\n')

riss_df = pd.DataFrame(real_list, columns=['제목', '저자', '권호사항', '키워드', '수록면', '국문초록', '영문초록'])
riss_df.to_excel('220204_특수체육학회지_크롤링_2002_2006.xlsx')

riss_df


한국특수체육학회지는 1993년부터 2021년까지 89권이 출간되었습니다
따라서 링크의 개수는 89개 입니다 

1번째 링크입니다
한국특수체육학회지 Vol.14 No.4 논문의 갯수는 10개 입니다.
1번째 링크의 크롤링이 완료되었습니다 

2번째 링크입니다
한국특수체육학회지 Vol.14 No.3 논문의 갯수는 6개 입니다.
2번째 링크의 크롤링이 완료되었습니다 

3번째 링크입니다
한국특수체육학회지 Vol.14 No.2 논문의 갯수는 7개 입니다.
3번째 링크의 크롤링이 완료되었습니다 

4번째 링크입니다
한국특수체육학회지 Vol.14 No.1 논문의 갯수는 7개 입니다.
4번째 링크의 크롤링이 완료되었습니다 

5번째 링크입니다
한국특수체육학회지 Vol.13 No.4 논문의 갯수는 11개 입니다.
5번째 링크의 크롤링이 완료되었습니다 

6번째 링크입니다
한국특수체육학회지 Vol.13 No.3 논문의 갯수는 6개 입니다.
6번째 링크의 크롤링이 완료되었습니다 

7번째 링크입니다
한국특수체육학회지 Vol.13 No.2 논문의 갯수는 10개 입니다.
7번째 링크의 크롤링이 완료되었습니다 

8번째 링크입니다
한국특수체육학회지 Vol.13 No.1 논문의 갯수는 8개 입니다.
8번째 링크의 크롤링이 완료되었습니다 

9번째 링크입니다
한국특수체육학회지 Vol.12 No.4 논문의 갯수는 9개 입니다.
9번째 링크의 크롤링이 완료되었습니다 

10번째 링크입니다
한국특수체육학회지 Vol.12 No.3 논문의 갯수는 6개 입니다.
10번째 링크의 크롤링이 완료되었습니다 

11번째 링크입니다
한국특수체육학회지 Vol.12 No.2 논문의 갯수는 8개 입니다.
11번째 링크의 크롤링이 완료되었습니다 

12번째 링크입니다
한국특수체육학회지 Vol.12 No.1 논문의 갯수는 9개 입니다.
12번째 링크의 크롤링이 완료되었습니다 

13번째 링크입니다
한국특수체육학회지 Vol.11 No.3 논문의 갯수는 18개 입니다.
13번째 링크의 크롤링이 완료되었습니다 

14번

,제목,저자,권호사항,키워드,수록면,국문초록,영문초록
0,장애인ᆞ비장애인의 신체활동참여에 대한 예비생활체육지도자가 인지하고 있는 가치 ...,이범진 ( Bom Jin Lee ) ;...,Vol.14 No.4 [2006],장애인 생활체육; 생활체육지도자; 운동/신체활동 가치; Sport-For-All; ...,1-16(16쪽),\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t\t\n\t...,초록없음
1,2004 아테네 파랄림픽 휠체어럭비 경기의 승ᆞ패 요인에 관한 연구 ...,최현희 ( Hyun Hee Choi ),Vol.14 No.4 [2006],휠체어럭비; 사지마비; 판별분석; 턴오버; Wheelchair Rugby; Quad...,17-36(20쪽),\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t\t\n\t...,초록없음
2,한국판 통합교육에 대한 체육교육전공 대학생의 판단 설문지에 관한 타당성 검증 ...,손영수 ( Young Soo Son ) ...,Vol.14 No.4 [2006],통합교육; 교사의 판단; 태도; 비교 문화적 관점; 탐색적; 확인적 요인분석; In...,37-56(20쪽),\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t\t\n\t...,초록없음
3,"장애인 휠체어 농구선수와 비장애인의 건강관련체력, 골밀도 및 혈중지질 농도 비교 ...",최윤진 ( Youn Jin Choi ) ...,Vol.14 No.4 [2006],휠체어농구선수; 건강관련체력; 골밀도; 혈중지질 농도; Wheelchair-Depe...,57-75(19쪽),\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t\t\n\t...,초록없음
4,통합체육 수업매뉴얼(MTI)의 적용 및 분석(1) ...,이경준 ( Kyoung June Yi ) ...,Vol.14 No.4 [2006],통합체육; 수업매뉴얼; Inclusive Physical Education; Mov...,77-107(31쪽),\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t\t\n\t...,초록없음
...,...,...,...,...,...,...,...
149,휠체어농구 슛의 난이도 추정 = Difficu...,전혜자,Vol.11 No.1 [2003],Wheelchair basketball; Wheelchair sports; Item...,243-250(8쪽),The purpose of this study was to analyze diffi...,초록없음
150,정신지체 장애인과 일반 성인의 건강에 관련된 체력 비교 ...,오덕자,Vol.11 No.1 [2003],Mental retardation; Health related physical fi...,251-258(8쪽),The purpose of this study was to comparison of...,초록없음
151,운동참가 지체장애인의 자기결정동기와 고독감 및 생활만족도의 구조모형 분석 ...,오광진,Vol.11 No.1 [2003],Self-determination; Motivation; Loneliness; Li...,259-271(13쪽),This study was to analysis the structural equa...,초록없음
152,노인체조프로그램 참여가 뇌졸중으로 인한 거동장애노인의 신체조성과 생활동작능력에 미치...,김설향,Vol.11 No.1 [2003],정보없음,273-283(11쪽),The study examined the effect of the body comp...,초록없음
